In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from sklearn.metrics import accuracy_score, classification_report
import torch
from tqdm import tqdm

# Load Dataset
try:
    df = pd.read_csv('data /smart_watch_review.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: File not found.")

Dataset loaded successfully.


In [2]:
# Generate Ground Truth Labels
rating_col = None
for col in df.columns:
    if 'rating' in col.lower() or 'star' in col.lower():
        rating_col = col
        break

if rating_col:
    def extract_rating(val):
        try:
            return float(str(val).split()[0])
        except:
            return 3.0
    df['numeric_rating'] = df[rating_col].apply(extract_rating)
    
    def get_label(rating):
        if rating > 3: return 'Positive'
        elif rating < 3: return 'Negative'
        else: return 'Neutral'
    
    df['Sentiment'] = df['numeric_rating'].apply(get_label)
    print("Labels generated.")
else:
    print("Warning: No rating column found.")

Labels generated.


In [3]:
# Load Pre-trained Model (Product Reviews 1-5 stars)
MODEL = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
print("BERT Model Loaded.")

# Prediction Function
def predict_sentiment(text):
    try:
        encoded_text = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        output = model(**encoded_text)
        scores = output.logits[0].detach().numpy()
        scores = softmax(scores)
        
        # Labels: 0->1 star, 1->2 stars, 2->3 stars, 3->4 stars, 4->5 stars
        star_rating = np.argmax(scores) + 1
        
        if star_rating <= 2:
            return 'Negative'
        elif star_rating == 3:
            return 'Neutral'
        else:
            return 'Positive'
    except:
        return "Neutral"

BERT Model Loaded.


In [4]:
# Debug Data Extraction
print("Unique numeric ratings:", df['numeric_rating'].unique())
print("Unique sentiments:", df['Sentiment'].unique())

# Run Predictions (on a sample to save time, or full dataset)
print("Running predictions on first 500 rows for verification...")
sample_df = df.head(500).copy()
tqdm.pandas()
sample_df['Predicted'] = sample_df['review'].progress_apply(predict_sentiment)

# Evaluate
acc = accuracy_score(sample_df['Sentiment'], sample_df['Predicted'])
print(f"\nAccuracy on Sample: {acc*100:.2f}%")
print("\nClassification Report:\n", classification_report(sample_df['Sentiment'], sample_df['Predicted']))

# Show Misclassified Examples
print("\nMisclassified Examples:")
errors = sample_df[sample_df['Sentiment'] != sample_df['Predicted']].head(10)
for idx, row in errors.iterrows():
    print(f"Text: {row['review'][:100]}...")
    print(f"True: {row['Sentiment']} (Rating: {row['numeric_rating']}) | Pred: {row['Predicted']}")
    print("-" * 50)

Unique numeric ratings: [3. 2. 5. 1. 4.]
Unique sentiments: ['Neutral' 'Negative' 'Positive']
Running predictions on first 500 rows for verification...


100%|██████████| 500/500 [00:14<00:00, 34.05it/s]



Accuracy on Sample: 29.00%

Classification Report:
               precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       197
     Neutral       0.18      0.36      0.24       109
    Positive       0.37      0.55      0.44       194

    accuracy                           0.29       500
   macro avg       0.18      0.30      0.23       500
weighted avg       0.18      0.29      0.22       500


Misclassified Examples:
Text: Amazing smartwatch with excellent performance. The heart rate tracking is very accurate, and step co...
True: Neutral (Rating: 3.0) | Pred: Positive
--------------------------------------------------
Text: Product quality is good but the strap material could have been better. Features work fine and notifi...
True: Negative (Rating: 2.0) | Pred: Neutral
--------------------------------------------------
Text: The watch works fine but sometimes hangs while switching between apps. Battery performance varies a ...
True: Negative (Rat

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
